In [1]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [2]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [3]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

In [4]:
from agent import DQNAgent
from agent.model import ConvQModel
from agent.configuration import DQNConfig
from agent.exploration import LinearExploration
from agent.memory import PriorityMemory

In [5]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=True
)
target_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=True
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dueling_ddqn_p_gobang_n7.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True,
    memory=PriorityMemory(capacity=5000)
)
agent = DQNAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/dueling_ddqn_p_gobang_n7')

2023-05-05 05:15:48.654 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.005114840668818654, reward sum: -48
2023-05-05 05:16:00.113 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 25/50
2023-05-05 05:16:01.361 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.0017512259598701676, reward sum: -40
2023-05-05 05:16:02.651 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.0015012598606808734, reward sum: -48.2
2023-05-05 05:16:03.594 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.0021279856634690156, reward sum: -33
2023-05-05 05:16:04.297 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.0005735561098335594, reward sum: -24
2023-05-05 05:16:05.422 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.0007351693591203564, reward sum: -34
2023-05-05 05:16:06.554 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.0016389305096729627, reward sum: -33
2023-05-05 05:16:07.087 | 

In [6]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=True
)
target_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=True
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dueling_ddqn_p_gobang_n11.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/dueling_ddqn_p_gobang_n11')

2023-05-05 07:17:04.663 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.047772567418524176, reward sum: -87
2023-05-05 07:17:59.797 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 17/50
2023-05-05 07:18:01.908 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.04840002902371109, reward sum: -65
2023-05-05 07:18:03.853 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.02156633467560155, reward sum: -52
2023-05-05 07:18:07.379 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.03406061205167688, reward sum: -85
2023-05-05 07:18:10.373 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.033463470810761856, reward sum: -65
2023-05-05 07:18:14.185 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.027387136516809116, reward sum: -81
2023-05-05 07:18:16.914 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.025305913085455472, reward sum: -54
2023-05-05 07:18:19.619 | INFO     | 